In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# spark = SparkSession.builder.master('local[2]').getOrCreate()

from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db| d_deviceusageimpact|      false|
| tabd_db|dailyinternetusag...|      false|
| tabd_db|   deviceusageimpact|      false|
| tabd_db|globalinternetusa...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-----------+



In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.D_DeviceUsageImpact 
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.D_DeviceUsageImpact  (
        year INT,
        time_spend_mobile INT,
        time_spend_desktop INT,
        impact_mobile DOUBLE,
        impact_desktop DOUBLE, 
        avg_time_spend_mobile DOUBLE,
        avg_time_spend_desktop DOUBLE
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/D_DeviceUsageImpact/'
    """
)


DataFrame[]

In [4]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    
    """
).show()

+----------+-------+----+
|time_spend| device|year|
+----------+-------+----+
|        43|desktop|2011|
|        32| mobile|2011|
|        47|desktop|2012|
|        36| mobile|2012|
|        49|desktop|2013|
|        45| mobile|2013|
|        47|desktop|2014|
|        63| mobile|2014|
|        42|desktop|2015|
|        81| mobile|2015|
|        40|desktop|2016|
|        94| mobile|2016|
|        42|desktop|2017|
|       109| mobile|2017|
|        39|desktop|2018|
|       122| mobile|2018|
|        39|desktop|2019|
|       132| mobile|2019|
|        38|desktop|2020|
|       143| mobile|2020|
+----------+-------+----+
only showing top 20 rows



In [5]:
DeviceUsageImpact = spark.sql(
    """
    
    SELECT DISTINCT A.year, A.mobile as time_spend_mobile, B.desktop as time_spend_desktop, ((A.mobile/(A.mobile+B.desktop))*100) as impact_mobile, 
                                                ((B.desktop/(A.mobile+B.desktop))*100) as impact_desktop,  
                                                AM.avg_mobile/CM.count_mobile as avg_time_spend_mobile,
                                                AD.avg_desktop/CD.count_desktop as avg_time_spend_desktop

    FROM
    
    (SELECT DISTINCT year, device, SUM(time_spend) AS mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY year, device) as A
    
    INNER JOIN
    
    (SELECT DISTINCT year, device, SUM(time_spend) AS desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY year, device) as B 

    ON A.year = B.year
    
     INNER JOIN
    
    (SELECT DISTINCT device, SUM(time_spend) AS avg_mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY device) as AM 
    
    INNER JOIN
    
    (SELECT DISTINCT device, SUM(time_spend) AS avg_desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY device) as AD
    
    INNER JOIN
    
    (SELECT DISTINCT device, COUNT(time_spend) AS count_mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY device) as CM
    
    INNER JOIN
    
    (SELECT DISTINCT device, COUNT(time_spend) AS count_desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY device) as CD
        
    """
   
)

In [6]:
spark.sql(
    """
    
    SELECT DISTINCT A.year, A.mobile as time_spend_mobile, B.desktop as time_spend_desktop, ((A.mobile/(A.mobile+B.desktop))*100) as impact_mobile, 
                                                ((B.desktop/(A.mobile+B.desktop))*100) as impact_desktop,  
                                                AM.avg_mobile/CM.count_mobile as avg_time_spend_mobile,
                                                AD.avg_desktop/CD.count_desktop as avg_time_spend_desktop

    FROM
    
    (SELECT DISTINCT year, device, SUM(time_spend) AS mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY year, device) as A
    
    INNER JOIN
    
    (SELECT DISTINCT year, device, SUM(time_spend) AS desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY year, device) as B 

    ON A.year = B.year
    
     INNER JOIN
    
    (SELECT DISTINCT device, SUM(time_spend) AS avg_mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY device) as AM 
    
    INNER JOIN
    
    (SELECT DISTINCT device, SUM(time_spend) AS avg_desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY device) as AD
    
    INNER JOIN
    
    (SELECT DISTINCT device, COUNT(time_spend) AS count_mobile
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'mobile'
    
    GROUP BY device) as CM
    
    INNER JOIN
    
    (SELECT DISTINCT device, COUNT(time_spend) AS count_desktop
    FROM tabd_db.DailyInternetUsageWorldwideByDevice
    WHERE device = 'desktop'
    
    GROUP BY device) as CD
        
    """
   
).toPandas()

,year,time_spend_mobile,time_spend_desktop,impact_mobile,impact_desktop,avg_time_spend_mobile,avg_time_spend_desktop
0,2011,32,43,42.666667,57.333333,92.0,42.090909
1,2019,132,39,77.192982,22.807018,92.0,42.090909
2,2021,155,37,80.729167,19.270833,92.0,42.090909
3,2014,63,47,57.272727,42.727273,92.0,42.090909
4,2018,122,39,75.776398,24.223602,92.0,42.090909
5,2012,36,47,43.373494,56.626506,92.0,42.090909
6,2013,45,49,47.872340,52.127660,92.0,42.090909
7,2020,143,38,79.005525,20.994475,92.0,42.090909
8,2017,109,42,72.185430,27.814570,92.0,42.090909
9,2015,81,42,65.853659,34.146341,92.0,42.090909


In [7]:
DeviceUsageImpact \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/warehouse/tabd.db/D_DeviceUsageImpact")